### Edgar Moises Hernandez-Gonzalez
#### 27-07-20-26-08-20
#### Clasificacion binaria de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFT (3D)
##### Entrada de 90x17

In [1]:
from google.colab import drive

In [15]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import numpy as np
import pandas as pd
import time
from scipy.signal import spectrogram
from sklearn.metrics import confusion_matrix
from keras.models import load_model

In [4]:
c3_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 0:768]
c4_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/MIEEG_test.csv", header=None).iloc[:, 1536:2304]

In [5]:
x_test = pd.concat([c3_test, c4_test], axis=1)

In [6]:
y_test = pd.read_csv("/content/drive/My Drive/BCI-II-III/Datos/etiquetas_test.csv", header=None)

In [7]:
print(x_test.shape)
print(y_test.shape)

(140, 1536)
(140, 1)


In [8]:
#unir 2 canales y 3 segundos
def unir_espectrogramas_vertical(data):
  fs = 128
  datos = np.zeros((data.shape[0],90,17))
  temporal = np.zeros((90,17))
  for i in range(data.shape[0]): #n muestras
    for j in range(2): #2 canales
      sig = data.iloc[i, j*768:(j+1)*768]
      sig = sig[32:416]
      #espectrograma de STFFT
      f, t, Sxx = spectrogram(sig, fs=128, window='hann', nperseg=128,
                              noverlap=112, nfft=256, scaling='spectrum')
      temporal[j*45:(j+1)*45, :] = Sxx[16:61, :]
    datos[i] = temporal
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [9]:
#llamar a unir_espectrogramas_vertical(data)
x_test = unir_espectrogramas_vertical(x_test)

0
100


In [10]:
print(x_test.shape)

(140, 90, 17)


In [11]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(140, 90, 17, 1)


In [12]:
model = load_model('/content/drive/My Drive/BCI-II-III/Modelos/STFT_CNN.hdf5')

In [13]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 90, 17, 64)        640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 45, 8, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 45, 8, 64)         36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 22, 4, 64)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 5632)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               721024    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [19]:
inicio = time.time()

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.17541933059692383


In [ ]:
print("loss: %.4f" % test_loss)
print("accuracy: %.4f" % test_acc)

loss: 0.4275
accuracy: 0.8286


In [ ]:
probabilidades = model.predict(x_test)
y_pred = np.argmax(probabilidades, 1) #1=fila

In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("confusion matrix:\n", matriz_confusion)

confusion matrix:
 [[59 11]
 [13 57]]


In [21]:
inicio = time.time()

probabilidades = model.predict(x_test[0].reshape(1,90,17,1))
y_pred = np.argmax(probabilidades, 1) #1=fila

fin = time.time()
print("Tiempo:", fin - inicio)

Tiempo: 0.046041250228881836
